[![Open In Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-colab.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/structural-break/quickstarters/random-submission/random-submission.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/master/competitions/structural-break/assets/banner.webp)

# Setup

The first steps to get started are:
1. Get the setup command
2. Execute it in the cell below

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/reveal-token.gif)

In [5]:
%pip install crunch-cli --upgrade --quiet --progress-bar off
!crunch setup structural-break thirsty-orangutan --token qlyGnbC26jCPE1ttQFbBDK4t


crunch-cli, version 7.3.0
no quickstarter available, leaving directory empty
data/X_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/y_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
                                
---
Success! Your environment has been correctly setup.
Next recommended actions:
 - To get inside your workspace directory, run: cd structural-break-thirsty-orangutan
 - To see all of the available commands of the CrunchDAO CLI, run: crunch --h

In [9]:
%cd structural-break-thirsty-orangutan

/content/structural-break-thirsty-orangutan


In [1]:
# # Install the Crunch CLI
# %pip install --upgrade crunch-cli

# # Setup your local environment
# !crunch setup --notebook structural-break hello --token aaaabbbbccccddddeeeeffff

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requirements-parser
    Found existing installation: requirements-parser 0.9.0
    Uninstalling requirements-parser-0.9.0:
      Successfully uninstalled requirements-parser-0.9.0
crunch-cli, version 7.3.0

---
Your token seems to have expired or is invalid.

Please follow this link to copy and paste your new setup command:
https://hub.crunchdao.com/competitions/structural-break/submit

If you think that is an error, please contact an administrator.


# Your model

## Setup

In [6]:
import os
import random
import typing

# Import your dependencies
import joblib
import pandas as pd
import sklearn.metrics

In [7]:
import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>

cli version: 7.3.0
available ram: 12.67 gb
available cpu: 2 core
----


## Data

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [10]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

data/X_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


### `X_train`

Index:
- `id`: the ID of the dataset
- `time`: arbitrary amount of time sampled regularely

Columns:
- `value`: the timeseries data
- `period`: if you are in an **initial segment** (0) or an **extension segment** (1)

In [11]:
X_train

value  period
id    time                  
0     0    -0.005564       0
      1     0.003705       0
      2     0.013164       0
      3     0.007151       0
      4    -0.009979       0
...              ...     ...
10000 2134  0.001137       1
      2135  0.003526       1
      2136  0.000687       1
      2137  0.001640       1
      2138  0.001074       1

[23715734 rows x 2 columns]

### `y_train`

This is a simple `pandas.Series` that tells if a dataset id has a structural breakpoint or not.

Index:
- `id`: the ID of the dataset

Value:
- `structural_breakpoint`: the value you need to predict

In [12]:
y_train

,structural_breakpoint
id,
0,False
1,False
2,True
3,False
4,False
...,...
9996,False
9997,False
9998,False


### `X_test`

This is a **`list` of `pandas.DataFrame`** that have the same format as [`X_train`](#X_train).

It is provided as a list to make sure you are encouraged to read the records **one by one**, __as this will be mandatory in the [`infer()`](#infer) function__.

In [13]:
print("Number of datasets:", len(X_test))

Number of datasets: 101


In [14]:
X_test[0]

value  period
id    time                  
10001 0     0.010753       0
      1    -0.031915       0
      2    -0.010989       0
      3    -0.011111       0
      4     0.011236       0
...              ...     ...
      2774 -0.013937       1
      2775 -0.015649       1
      2776 -0.009744       1
      2777  0.025375       1
      2778 -0.001515       1

[2779 rows x 2 columns]

## Implementation

### `train()`

In the training function, users build and train the model to make inferences on the test data. <br />
Your model must be stored in the `model_directory_path`.

In [ ]:
def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
):
    model = ...

    joblib.dump(model, os.path.join(model_directory_path, 'model.joblib'))

### `infer()`

In the inference function, the trained model is loaded and used to make inferences on a sample of data that matches the characteristics of the training test.

#### Setup

Once your model is loaded, you must do a `yield` to signal it to the runner. <br />
After that you can start reading data from `X_test`.

#### Iteration

The datasets must be read **one by one** and each value must be returned with a `yield <value>`. <br />
If you try to skip this, you will get an error. <br />
All values are then concatenated into a prediction file.

**Warning: The datasets can only be iterated once!**

#### Cleanup

Code can be executed after the `for` loop if you need to persist state or do some cleanup.

In [ ]:
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

    yield  # mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
        # prediction = model.predict(dataset)
        prediction = round(random.random(), 2)

        yield prediction  # send the prediction for the current dataset

## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [ ]:
crunch.test(
    # Uncomment to disable the train
    # force_first_train=False,

    # Uncomment to disable the determinism check
    # no_determinism_check=True,
)

## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"].astype(float)

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)